# Importation des bibliothèques

In [1]:
# importation des bibliothèques
import matplotlib.pyplot as plt #pour la visualisation graphique
import pandas as pd #pour la lecture du fichier excel, en l'occurence la base des données
from sklearn.model_selection import train_test_split #pour la partion des données en données de test et d'entrainnement
from sklearn.preprocessing import StandardScaler, MinMaxScaler #respectivement pour la standardisation et la normalisation des données
from sklearn.metrics import mean_squared_error, r2_score #essentiels dans l'évaluation du modèle
from ydata_profiling import ProfileReport #pour l'analyse des données brutes
from tensorflow.keras.models import Sequential #Essentiel pour la création du réseau des neurones
from tensorflow.keras.layers import Dense #Pour les couches du réseau
from tensorflow.keras.utils import plot_model 

# Chargement des données

In [ ]:
# chargement des données depuis la destination dans le PC
donnees = pd.read_excel("energy_efficiency.xlsx")

In [ ]:
# Apperçu des cinq premières lignes des données
donnees.head()

In [ ]:
# Description des données
donnees.describe;

In [ ]:
# Informations diverses sur les données
donnees.info;

# Analyse des données avec ProfileReport

In [ ]:
profile = ProfileReport(donnees, title = 'Profiling Report')
profile

# Opérations sur les données

In [ ]:
# Définition des features et des targets entre les données

X = donnees.iloc[:, :8]
y = donnees.iloc[:, 8:]

In [ ]:
# Partition des données en données de test et données d'entrainnement

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 50)

In [ ]:
# Standardisation des donneés
scaler = StandardScaler()

# Standardisation des données d'entrainnement
X_train = scaler.fit_transform(X_train)

# Standardisation  des données de test
X_test = scaler.fit_transform(X_test)

#NORMALISATION DES DONNEES
normalizer = MinMaxScaler()

# Normalisation des données d'entrainnement
X_train = normalizer.fit_transform(X_train)

# Normalisation des données de test
X_test = normalizer.fit_transform(X_test)

# Choix et entrainnement du modèle

In [ ]:
# Initialisation du modèle de réseau des neurones
import random
random.seed(400)

model = Sequential()

model.add(Dense(200, activation = 'tanh', input_dim = X_train.shape[1])) #Couche d'entrée avec 200 neuronnes et utilisation de la fonction d'activation "tangente hyperbolique (tanh)"

model.add(Dense(120, activation = 'tanh')) #Couche cachée avec 120 neurones

model.add(Dense(60, activation = 'tanh')) # couche cachée avec 60 neurones

model.add(Dense(y_train.shape[1])) #Couche de sortie

In [ ]:
# Compilation du modèle
model.compile(optimizer = 'adam', loss = 'mean_squared_error') #Utilisation de l'optimiseur Adam sur la fonction de perte MSE

In [ ]:
# Entrainnement du modèle
model.fit(X_train, y_train, epochs = 500, batch_size = 5, verbose = 0) #Entrainnement sur 500 époques avec des lots de 5 éléments

# Prédiction des cibles avec le modèle choisi

In [ ]:
# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)
y_pred

# Evaluation des performances du modèle

In [ ]:
# Erreur quadratique moyenne
mse = mean_squared_error(y_test, y_pred)
print("Erreur quadratique moyenne (MSE) =",mse)

# Coefficient de détermination
r2 = r2_score(y_pred, y_test)
print("Coefficient de détermination (r2) =",r2)

# Visualisation graphique

In [ ]:
# Visualisation de l'écart entre les prédictions et les valeurs réelles
plt.figure(figsize = (12, 5))

# Vérification des dimensions
print("y_test shape:",y_test.shape)
print("y_pred shape:",y_pred.shape)

# Conversion de DataFrames Pandas vers Numpy
if isinstance(y_test,pd.DataFrame):
    y_test = y_test.to_numpy()
if isinstance(y_pred,pd.DataFrame):
    y_pred = y_pred.to_numpy()

#Visualisation de la première sortie
plt.subplot(1, 2, 1)
plt.scatter(y_test[:, 0], y_pred[:, 0], color = 'blue', label = 'Prédictions')
plt.plot([y_test[:, 0].min(), y_test[:, 0].max()], [y_test[:, 0].min(), y_test[:, 0].max()],color = 'red', linestyle = '--', label = 'Ligne de référence') #ligne de référence
plt.title('Prédictions vs Réalité (Charge de chauffage)')
plt.xlabel('Valeurs réelles')
plt.ylabel('Valeurs prédites')
plt.legend()
plt.grid()

#Visualisation de la deuxième sortie
plt.subplot(1, 2, 2)
plt.scatter(y_test[:, 1], y_pred[:, 1], color = 'green', label = 'Prédictions')
plt.plot([y_test[:, 1].min(), y_test[:, 1].max()], [y_test[:, 1].min(), y_test[:, 1].max()],color = 'red', linestyle = '--', label = 'Ligne de référence') #ligne de référence
plt.title('Prédictions vs Réalité (Charge de refroidissement)')
plt.xlabel('Valeurs réelles')
plt.ylabel('Valeurs prédites')
plt.legend()
plt.grid()

plt.tight_layout()
plt.savefig('predictions_plot.png') #Enregistrement du graphique
plt.close() #Fermeture de la figure pour libérer la mémoire

In [ ]:
# Visualisation de l'image enregistrée
from PIL import Image
img = Image.open('predictions_plot.png') # ouverture du fichier image
img.show() # affichage de l'image
img

# Enregistrement du modèle entrainné

In [ ]:
# Enregistrement du modèle avec pickle

import pickle # on importe la librerie pickle pour l'enregistrement du modèle

saved_model=pickle.dumps(model) # "model" est le nom du model de réseau des neurones

model_de_pick=pickle.loads(saved_model)

In [ ]:
# Enregistrement du modèle (comme fichier partageable) avec joblib

import joblib

from joblib import Parallel, delayed

joblib.dump(model,"FinProjAIBac3.pkl")

# Vérification de l'existence du modèle enregistré

In [ ]:
# Chargement du modèle depuis le PC
model_neurones = joblib.load("FinProjAIBac3.pkl")

In [ ]:
# Prédictions avec le model chargé
y_pred_verif = model_neurones.predict(X_test)
y_pred_verif

In [ ]:
# Evaluation du modèle enregistré

# Erreur quadratique moyenne (MSE)
mse_verif = mean_squared_error(y_pred_verif, y_test)
print(f"Erreur quadratique moyenne (MSE) = {mse_verif}")

# Coefficient de détermination (r2)
r2_verif = r2_score(y_pred_verif, y_test)
print(f"Coefficient de détermination (r2) = {r2_verif}")

In [ ]:
# Matrice des écarts

import numpy as np
matrice_verif = y_pred_verif - y_pred #si cette matrice est nulle alors il n'y a pas d'écart entre les deux prédictions
matrice_nulle = np.all(matrice_verif == 0)
if matrice_nulle:
    print("La matrice est nulle")
else:
    print("La matrice n'est pas nulle")

In [ ]:
# Rapport des MSE et r2 pour les deux prédictions

rap_mse = mse/mse_verif
print(f"Le rapport des MSE est : {rap_mse}")

rap_r2 = r2/r2_verif
print(f"Le rapport des r2 est : {rap_r2}")